In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import requests
df = pd.read_csv("trip.csv")
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat), crs=4326)
df

,name,date,lng,lat,geometry
0,Papatoetoe,2024-09-29,174.855370,-36.977993,POINT (174.85537 -36.97799)
1,Taupo DeBretts,2024-10-01,176.099186,-38.703164,POINT (176.09919 -38.70316)
2,Ōtaki,2024-10-02,175.152154,-40.754474,POINT (175.15215 -40.75447)
3,Wellington TOP 10 Holiday Park,2024-10-07,174.909526,-41.236709,POINT (174.90953 -41.23671)
4,Taihape,2024-10-08,175.798334,-39.676280,POINT (175.79833 -39.67628)
5,Rydges Rotorua,2024-10-11,176.253875,-38.148658,POINT (176.25388 -38.14866)
6,Papatoetoe,2024-10-11,174.855370,-36.977993,POINT (174.85537 -36.97799)


In [20]:
leg_names = []
for i in range(len(df)-1):
    leg_names.append(df.name[i] + " -> " + df.name[i+1])
leg_names

['Papatoetoe -> Taupo DeBretts',
 'Taupo DeBretts -> Ōtaki',
 'Ōtaki -> Wellington TOP 10 Holiday Park',
 'Wellington TOP 10 Holiday Park -> Taihape',
 'Taihape -> Rydges Rotorua',
 'Rydges Rotorua -> Papatoetoe']

In [2]:
coords = ";".join(df.lng.astype(str) + "," + df.lat.astype(str))
coords

'174.8553697,-36.9779932;176.0991857,-38.7031645;175.1521538,-40.7544744;174.9095263,-41.2367087;175.7983341,-39.6762799;176.2538754,-38.1486577;174.8553697,-36.9779932'

In [3]:
r = requests.get("http://router.project-osrm.org/route/v1/driving/" + coords, params={
    "geometries": "geojson",
    "steps": "true",
    "annotations": "distance,duration"
})
r

<Response [200]>

In [14]:
legs = r.json()["routes"][0]["legs"]
print(len(legs))
parsed_legs = []
for leg in legs:
    leg_coords = []
    for step in leg["steps"]:
        leg_coords.extend(step["geometry"]["coordinates"])
    parsed_legs.append({"geometry": LineString(leg_coords), "duration": leg["duration"], "distance": leg["distance"]})

6


In [21]:
legs = gpd.GeoDataFrame(parsed_legs, geometry="geometry", crs=4326).merge(df.drop(columns="geometry"), left_index=True, right_index=True)
legs["leg_name"] = leg_names
legs

,geometry,duration,distance,name,date,lng,lat,leg_name
0,"LINESTRING (174.85537 -36.978, 174.8565 -36.97...",11521.5,257697.4,Papatoetoe,2024-09-29,174.855370,-36.977993,Papatoetoe -> Taupo DeBretts
1,"LINESTRING (176.0992 -38.70316, 176.09916 -38....",17088.0,340726.9,Taupo DeBretts,2024-10-01,176.099186,-38.703164,Taupo DeBretts -> Ōtaki
2,"LINESTRING (175.15217 -40.75449, 175.1521 -40....",3673.4,69198.7,Ōtaki,2024-10-02,175.152154,-40.754474,Ōtaki -> Wellington TOP 10 Holiday Park
3,"LINESTRING (174.9095 -41.23673, 174.90947 -41....",11073.7,225119.6,Wellington TOP 10 Holiday Park,2024-10-07,174.909526,-41.236709,Wellington TOP 10 Holiday Park -> Taihape
4,"LINESTRING (175.79831 -39.67629, 175.7983 -39....",12997.3,279674.8,Taihape,2024-10-08,175.798334,-39.676280,Taihape -> Rydges Rotorua
5,"LINESTRING (176.25392 -38.14866, 176.25392 -38...",9707.1,211036.3,Rydges Rotorua,2024-10-11,176.253875,-38.148658,Rydges Rotorua -> Papatoetoe


In [22]:
legs.to_file("trip.geojson", driver="GeoJSON")